In [1]:
'''# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session'''

'# This Python 3 environment comes with many helpful analytics libraries installed\n# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python\n# For example, here\'s several helpful packages to load\n\nimport numpy as np # linear algebra\nimport pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\n\n# Input data files are available in the read-only "../input/" directory\n# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory\n\nimport os\nfor dirname, _, filenames in os.walk(\'/kaggle/input\'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n\n# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" \n# You can also write temporary files to /kaggle/temp/, but they won\'t be saved outside of the current session'

## Importing the training text dataset

In [1]:
import pandas as pd
train_text=pd.read_csv('/kaggle/input/visual-taxonomy/train.csv')
train_text.head()

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,Men Tshirts,5,default,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN
1,1,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
2,2,Men Tshirts,5,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
3,3,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN
4,4,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN


In [2]:
train_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70213 entries, 0 to 70212
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        70213 non-null  int64 
 1   Category  70213 non-null  object
 2   len       70213 non-null  int64 
 3   attr_1    51867 non-null  object
 4   attr_2    55192 non-null  object
 5   attr_3    54698 non-null  object
 6   attr_4    59888 non-null  object
 7   attr_5    56493 non-null  object
 8   attr_6    38116 non-null  object
 9   attr_7    41415 non-null  object
 10  attr_8    37474 non-null  object
 11  attr_9    33565 non-null  object
 12  attr_10   24999 non-null  object
dtypes: int64(2), object(11)
memory usage: 7.0+ MB


## Importing the category attributes file

In [3]:
categort_attr=pd.read_parquet('/kaggle/input/visual-taxonomy/category_attributes.parquet', engine='pyarrow')
categort_attr

,Category,No_of_attribute,Attribute_list
0,Men Tshirts,5,"[color, neck, pattern, print_or_pattern_type, ..."
1,Sarees,10,"[blouse_pattern, border, border_width, color, ..."
2,Kurtis,9,"[color, fit_shape, length, occasion, ornamenta..."
3,Women Tshirts,8,"[color, fit_shape, length, pattern, print_or_p..."
4,Women Tops & Tunics,10,"[color, fit_shape, length, neck_collar, ocassi..."


In [4]:
category_len={'Men Tshirts':5,'Sarees':10,'Kurtis':9,'Women Tshirts':8,'Women Tops & Tunics':10}

## Importing the test dataset

In [5]:
test_text=pd.read_csv('/kaggle/input/visual-taxonomy/test.csv')
test_text.head()

,id,Category
0,0,Men Tshirts
1,1,Men Tshirts
2,2,Men Tshirts
3,3,Men Tshirts
4,4,Men Tshirts


## Working on the image data before modelling

### Importing Vision Transformer (ViT) from huggingface

In [6]:
from transformers import TFAutoModel, ViTFeatureExtractor

# Load the ViT feature extractor and model
vit_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = TFAutoModel.from_pretrained('google/vit-base-patch16-224-in21k')

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


The vit_extractor is an instance of ViTFeatureExtractor from the HuggingFace transformers library. It is designed to preprocess the images into a format that is suitable for the pre-trained Vision Transformer model.

### Importing the training images and preprocessing them to give them as input to ViT Extractor (Image Preprocessing & Feature Extraction)

In [7]:
import tensorflow as tf
import os
import numpy as np
import hashlib

def process_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, [224, 224])
    image = vit_extractor(images=image.numpy(), return_tensors="tf")['pixel_values']
    return image

def hash_features(features):
    return hashlib.sha256(features.tobytes()).hexdigest()

# Load existing features from .npy file
loaded_array = np.load('/kaggle/input/feature-npy-files/image_features_10k.npy')
loaded_hashes = set(hash_features(feature) for feature in loaded_array)

image_features = []
folder_path = '/kaggle/input/visual-taxonomy/train_images'
count=0
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        if count%1000==0: 
            print('Finished:',count)
        img_path = os.path.join(folder_path, filename)
        
        # Preprocess image
        image_tensor = process_image(img_path)
        
        # Extract image features using ViT
        with tf.device('/GPU:0'):
            outputs = vit_model(image_tensor)
        
        # Extract CLS token embeddings (representing the entire image)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token
        
        # Hash the features
        feature_hash = hash_features(cls_embedding.numpy())
        
        # Check if this image's features already exist in the loaded_array
        if feature_hash in loaded_hashes:
            continue
        
        # Append new features and update hash set
        image_features.append(cls_embedding.numpy())
        loaded_hashes.add(feature_hash)  # Add new hash to the set

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Finished: 0
Fini

KeyboardInterrupt: 

What is the CLS Token?
In the Vision Transformer (ViT) architecture, an image is split into patches, and these patches are treated as tokens that are fed into the transformer.
A special token, known as the [CLS] token (classification token), is prepended to the sequence of image patch tokens.
As the image passes through the transformer layers, the CLS token aggregates information from all the other tokens (patches). By the end of the model, the CLS token contains a global representation of the entire image.

In [8]:
# Convert the list of features to a NumPy array for modeling
image_features = np.array(image_features)
print(image_features.shape)

(14157, 1, 768)


In [9]:
# Save the new features
np.save('new_image_features.npy', image_features)
new_loaded_array=np.load('new_image_features.npy') 

# Flatten the image features to (32, 768) for each sample
new_loaded_array = new_loaded_array.reshape((new_loaded_array.shape[0], -1))
new_loaded_array.shape

(14157, 768)

In [11]:
#Concate the prev and new features
loaded_array= loaded_array.reshape((loaded_array.shape[0], -1))
concatenated=np.concatenate((loaded_array,new_loaded_array),axis=0)
concatenated.shape

(23666, 768)

In [12]:
np.save('image_features_23k.npy', concatenated)

## Training a combined model 

In [13]:
image_features=concatenated

In [14]:
# Checking for NaNs in the relevant columns
print(train_text[['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5', 'attr_6', 'attr_7', 'attr_8', 'attr_9', 'attr_10']].isna().sum())

attr_1     18346
attr_2     15021
attr_3     15515
attr_4     10325
attr_5     13720
attr_6     32097
attr_7     28798
attr_8     32739
attr_9     36648
attr_10    45214
dtype: int64


In [15]:
train_text = train_text.fillna('dummy_value')

In [16]:
# Calculate number of unique classes for each attribute
num_attr_1_classes = train_text['attr_1'].nunique()
num_attr_2_classes = train_text['attr_2'].nunique()
num_attr_3_classes = train_text['attr_3'].nunique()
num_attr_4_classes = train_text['attr_4'].nunique()
num_attr_5_classes = train_text['attr_5'].nunique()
num_attr_6_classes = train_text['attr_6'].nunique()
num_attr_7_classes = train_text['attr_7'].nunique()
num_attr_8_classes = train_text['attr_8'].nunique()
num_attr_9_classes = train_text['attr_9'].nunique()
num_attr_10_classes = train_text['attr_10'].nunique()

In [17]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import numpy as np

# Function to encode labels and convert to correct dtype
def encode_labels(column):
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(train_text[column].fillna('dummy_value'))
    onehot_encoded = to_categorical(integer_encoded)
    onehot_encoded = onehot_encoded.astype('float32')  # Ensure the correct dtype
    return onehot_encoded, label_encoder

# Encode the labels for each attribute
attr_1_labels, encoder_attr_1 = encode_labels('attr_1')
attr_2_labels, encoder_attr_2 = encode_labels('attr_2')
attr_3_labels, encoder_attr_3 = encode_labels('attr_3')
attr_4_labels, encoder_attr_4 = encode_labels('attr_4')
attr_5_labels, encoder_attr_5 = encode_labels('attr_5')
attr_6_labels, encoder_attr_6 = encode_labels('attr_6')
attr_7_labels, encoder_attr_7 = encode_labels('attr_7')
attr_8_labels, encoder_attr_8 = encode_labels('attr_8')
attr_9_labels, encoder_attr_9 = encode_labels('attr_9')
attr_10_labels, encoder_attr_10 = encode_labels('attr_10')


In [18]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Category' column
train_text['Category'] = label_encoder.fit_transform(train_text['Category'])

In [19]:
# Flatten the image features to (32, 768) for each sample
image_features = image_features.reshape((image_features.shape[0], -1))

# Redefine the model with correct input shape
from tensorflow.keras.layers import Input, Dense, Concatenate, Flatten
from tensorflow.keras.models import Model

# Input layers
image_input = Input(shape=(768,), name='image_input')  # For ViT features
tabular_input = Input(shape=(2,), name='tabular_input')  # For tabular data (2 features)

# Dense layers for image features
x1 = Dense(128, activation='relu')(image_input)

# Dense layers for tabular features
x2 = Dense(128, activation='relu')(tabular_input)

# Concatenate image and tabular inputs
x = Concatenate()([x1, x2])

# Further dense layers after concatenation
x = Dense(128, activation='relu')(x)

# Output layers for attributes
output_1 = Dense(num_attr_1_classes, activation='softmax', name='attr_1')(x)
output_2 = Dense(num_attr_2_classes, activation='softmax', name='attr_2')(x)
output_3 = Dense(num_attr_3_classes, activation='softmax', name='attr_3')(x)
output_4 = Dense(num_attr_4_classes, activation='softmax', name='attr_4')(x)
output_5 = Dense(num_attr_5_classes, activation='softmax', name='attr_5')(x)
output_6 = Dense(num_attr_6_classes, activation='softmax', name='attr_6')(x)
output_7 = Dense(num_attr_7_classes, activation='softmax', name='attr_7')(x)
output_8 = Dense(num_attr_8_classes, activation='softmax', name='attr_8')(x)
output_9 = Dense(num_attr_9_classes, activation='softmax', name='attr_9')(x)
output_10 = Dense(num_attr_10_classes, activation='softmax', name='attr_10')(x)

# Build the model
model = Model(inputs=[image_input, tabular_input], 
              outputs=[output_1, output_2, output_3, output_4, output_5, 
                       output_6, output_7, output_8, output_9, output_10])

# Compile the model
model.compile(optimizer='adam', 
              loss={'attr_1': 'categorical_crossentropy',
                    'attr_2': 'categorical_crossentropy',
                    'attr_3': 'categorical_crossentropy',
                    'attr_4': 'categorical_crossentropy',
                    'attr_5': 'categorical_crossentropy',
                    'attr_6': 'categorical_crossentropy',
                    'attr_7': 'categorical_crossentropy',
                    'attr_8': 'categorical_crossentropy',
                    'attr_9': 'categorical_crossentropy',
                    'attr_10': 'categorical_crossentropy'},
              metrics={'attr_1': 'accuracy',
                       'attr_2': 'accuracy',
                       'attr_3': 'accuracy',
                       'attr_4': 'accuracy',
                       'attr_5': 'accuracy',
                       'attr_6': 'accuracy',
                       'attr_7': 'accuracy',
                       'attr_8': 'accuracy',
                       'attr_9': 'accuracy',
                       'attr_10': 'accuracy'})  # Metrics for each output

# Train the model
history = model.fit([image_features, train_text.iloc[:, :2].values], 
                    [attr_1_labels, attr_2_labels, attr_3_labels, 
                     attr_4_labels, attr_5_labels, attr_6_labels, 
                     attr_7_labels, attr_8_labels, attr_9_labels, 
                     attr_10_labels],
                    epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50


I0000 00:00:1729147075.727919     108 service.cc:145] XLA service 0x5b8ced53a3d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729147075.727982     108 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1729147075.727992     108 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 37/592 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - attr_10_accuracy: 0.4410 - attr_1_accuracy: 0.0852 - attr_2_accuracy: 0.1080 - attr_3_accuracy: 0.1715 - attr_4_accuracy: 0.0986 - attr_5_accuracy: 0.1653 - attr_6_accuracy: 0.6456 - attr_7_accuracy: 0.4745 - attr_8_accuracy: 0.3940 - attr_9_accuracy: 0.1287 - loss: 5023.4663                   

I0000 00:00:1729147126.312016     108 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


592/592 ━━━━━━━━━━━━━━━━━━━━ 90s 58ms/step - attr_10_accuracy: 0.5751 - attr_1_accuracy: 0.2747 - attr_2_accuracy: 0.2488 - attr_3_accuracy: 0.2569 - attr_4_accuracy: 0.1834 - attr_5_accuracy: 0.3185 - attr_6_accuracy: 0.6777 - attr_7_accuracy: 0.5106 - attr_8_accuracy: 0.5234 - attr_9_accuracy: 0.3336 - loss: 879.5846 - val_attr_10_accuracy: 0.8215 - val_attr_1_accuracy: 0.0891 - val_attr_2_accuracy: 0.4651 - val_attr_3_accuracy: 0.5104 - val_attr_4_accuracy: 0.2824 - val_attr_5_accuracy: 0.0712 - val_attr_6_accuracy: 0.7334 - val_attr_7_accuracy: 0.5116 - val_attr_8_accuracy: 0.7360 - val_attr_9_accuracy: 0.1318 - val_loss: 236.2841
Epoch 2/50
592/592 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - attr_10_accuracy: 0.6033 - attr_1_accuracy: 0.2361 - attr_2_accuracy: 0.2765 - attr_3_accuracy: 0.2316 - attr_4_accuracy: 0.1849 - attr_5_accuracy: 0.3426 - attr_6_accuracy: 0.6689 - attr_7_accuracy: 0.5534 - attr_8_accuracy: 0.4975 - attr_9_accuracy: 0.4170 - loss: 50.0490 - val_attr_10_accuracy: 0.82

## Test the model

In [20]:
import tensorflow as tf
import os
import numpy as np
import hashlib

# Function to hash image features
def hash_feature(feature_array):
    feature_bytes = feature_array.tobytes()  # Convert the numpy array to bytes
    return hashlib.md5(feature_bytes).hexdigest()  # Return the MD5 hash of the feature

# Precomputed features array (loaded from the saved file)
test_loaded_array = np.load('/kaggle/input/feature-npy-files/test_features_8k.npy')  

# Compute hashes for all features in loaded_array
test_loaded_hashes = set([hash_feature(test_loaded_array[i]) for i in range(test_loaded_array.shape[0])])

# Function to preprocess image (as used in your code)
def process_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    image = tf.image.resize(image, [224, 224])
    image = vit_extractor(images=image.numpy(), return_tensors="tf")['pixel_values']
    return image

# Extract features for new images while skipping duplicates
test_features = []
folder_path = '/kaggle/input/visual-taxonomy/test_images'
count = 0

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"): 
        count += 1
        if count%1000:  # Stop after processing 10,000 images
            print('Finished',count)
        
        img_path = os.path.join(folder_path, filename)
        
        # Preprocess image
        image_tensor = process_image(img_path) 
        
        # Extract image features using ViT
        with tf.device('/GPU:0'):  # Using GPU
            outputs = vit_model(image_tensor)
        
        # Extract the CLS token embeddings (representing the entire image)
        cls_embedding = outputs.last_hidden_state[:, 0, :].numpy()  # CLS token
        
        # Compute the hash of the extracted feature
        feature_hash = hash_feature(cls_embedding)
        
        # Check if this feature already exists in loaded_array
        if feature_hash not in test_loaded_hashes:
            # Append the feature to the list if it's not a duplicate
            test_features.append(cls_embedding)
            # Add the new hash to the loaded_hashes to avoid duplicates in future
            test_loaded_hashes.add(feature_hash)

Finished 1
Finished 2
Finished 3
Finished 4
Finished 5
Finished 6
Finished 7
Finished 8
Finished 9
Finished 10
Finished 11
Finished 12
Finished 13
Finished 14
Finished 15
Finished 16
Finished 17
Finished 18
Finished 19
Finished 20
Finished 21
Finished 22
Finished 23
Finished 24
Finished 25
Finished 26
Finished 27
Finished 28
Finished 29
Finished 30
Finished 31
Finished 32
Finished 33
Finished 34
Finished 35
Finished 36
Finished 37
Finished 38
Finished 39
Finished 40
Finished 41
Finished 42
Finished 43
Finished 44
Finished 45
Finished 46
Finished 47
Finished 48
Finished 49
Finished 50
Finished 51
Finished 52
Finished 53
Finished 54
Finished 55
Finished 56
Finished 57
Finished 58
Finished 59
Finished 60
Finished 61
Finished 62
Finished 63
Finished 64
Finished 65
Finished 66
Finished 67
Finished 68
Finished 69
Finished 70
Finished 71
Finished 72
Finished 73
Finished 74
Finished 75
Finished 76
Finished 77
Finished 78
Finished 79
Finished 80
Finished 81
Finished 82
Finished 83
Finished 84
F

KeyboardInterrupt: 

In [21]:
# Convert the list of features to a NumPy array for testing
test_features = np.array(test_features)

predicted_values=len(test_features)
len(test_features)

13798

In [27]:
test_features= test_features.reshape((test_features.shape[0], -1))
test_features.shape

(13798, 768)

In [24]:
test_loaded_array= test_loaded_array.reshape((test_loaded_array.shape[0], -1))
test_loaded_array.shape

(8092, 768)

In [29]:
test_concat=np.concatenate((test_loaded_array,test_features))
test_concat.shape

(21890, 768)

In [30]:
# Save the array to a .npy file
np.save('test_features_21k.npy', test_concat)

# To load it back
test_loaded_array = np.load('test_features_21k.npy')

In [32]:
test_features=test_loaded_array

#### direct importing

In [34]:
test_features=np.load('/kaggle/input/feature-npy-files/test_features_8k.npy') 
len(test_features)

8092

In [37]:
# Reshape the test_features to match the expected input shape
test_features = test_features.reshape(test_features.shape[0], test_features.shape[2])
test_features.shape

#### Preprocessing text

In [31]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Encode categorical columns
categorical_columns = test_text.select_dtypes(include=['object']).columns
label_encoder = LabelEncoder()
for i in categorical_columns:
    test_text[i] = label_encoder.fit_transform(test_text[i])

#No need to normalize the length column
'''# Normalize numerical columns
numerical_columns = ['attr_1', 'attr_2', ..., 'attr_10']
scaler = StandardScaler()
train_df[numerical_columns] = scaler.fit_transform(train_df[numerical_columns])'''



In [34]:
predicted_values=len(test_features)

# Convert to numpy arrays
test_tabular_features = test_text[:predicted_values].values

### Get predictions

In [35]:
# Get predictions from the trained model
predictions = model.predict([test_features, test_tabular_features])

685/685 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step


In [36]:
print("Shape of test_tabular_features:", test_tabular_features.shape)

Shape of test_tabular_features: (21890, 2)


In [37]:
# Extract the class with the highest probability for each attribute prediction
predicted_labels = []
for i, prediction in enumerate(predictions):
    predicted_labels.append(np.argmax(prediction, axis=1))

# Display the predicted labels for each attribute
for i, label in enumerate(predicted_labels):
    print(f"Predicted Labels for Attribute {i+1}: {label}")


Predicted Labels for Attribute 1: [7 7 7 ... 3 3 3]
Predicted Labels for Attribute 2: [ 7  7  7 ... 14 14 14]
Predicted Labels for Attribute 3: [10 10 10 ...  9  9  9]
Predicted Labels for Attribute 4: [ 2 12 12 ...  6  6  6]
Predicted Labels for Attribute 5: [10 10 10 ...  8  8  8]
Predicted Labels for Attribute 6: [1 1 1 ... 1 1 1]
Predicted Labels for Attribute 7: [2 2 2 ... 2 2 2]
Predicted Labels for Attribute 8: [ 2  2  2 ... 10 10 10]
Predicted Labels for Attribute 9: [4 4 4 ... 4 4 4]
Predicted Labels for Attribute 10: [2 2 2 ... 4 4 4]


## Submission 

In [38]:
import pandas as pd
import numpy as np

# Number of test samples
total_test_samples = 30205  # Total number of test rows
#predicted_samples = 4440    # Number of predicted samples

# Create a DataFrame with NaN for all rows initially
submission_df = pd.DataFrame(np.nan, index=range(total_test_samples), columns=['id','Category','len',
    'Attribute_1', 'Attribute_2', 'Attribute_3', 'Attribute_4', 'Attribute_5', 
    'Attribute_6', 'Attribute_7', 'Attribute_8', 'Attribute_9', 'Attribute_10'
])


In [39]:
test_text.shape

(30205, 2)

In [40]:
test_text=pd.read_csv('/kaggle/input/visual-taxonomy/test.csv')
test_text.head()

,id,Category
0,0,Men Tshirts
1,1,Men Tshirts
2,2,Men Tshirts
3,3,Men Tshirts
4,4,Men Tshirts


In [41]:
submission_df.iloc[:30205,0] = test_text.iloc[:,0]
submission_df.iloc[:30205,1] = test_text.iloc[:,1]

/tmp/ipykernel_30/4028425247.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Men Tshirts' 'Men Tshirts' 'Men Tshirts' ... 'Women Tops & Tunics'
 'Women Tops & Tunics' 'Women Tops & Tunics']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  submission_df.iloc[:30205,1] = test_text.iloc[:,1]


In [42]:
submission_df.loc[:, 'len'] = submission_df['Category'].map(category_len)

## Reversing the encoding

In [43]:
# Reverse the encoding of predicted labels using LabelEncoder's inverse_transform
reversed_attr_1 = encoder_attr_1.inverse_transform(predicted_labels[0])  # For Attribute 1
reversed_attr_2 = encoder_attr_2.inverse_transform(predicted_labels[1])  # For Attribute 2
reversed_attr_3 = encoder_attr_3.inverse_transform(predicted_labels[2])  # For Attribute 3
reversed_attr_4 = encoder_attr_4.inverse_transform(predicted_labels[3])  # For Attribute 4
reversed_attr_5 = encoder_attr_5.inverse_transform(predicted_labels[4])  # For Attribute 5
reversed_attr_6 = encoder_attr_6.inverse_transform(predicted_labels[5])  # For Attribute 6
reversed_attr_7 = encoder_attr_7.inverse_transform(predicted_labels[6])  # For Attribute 7
reversed_attr_8 = encoder_attr_8.inverse_transform(predicted_labels[7])  # For Attribute 8
reversed_attr_9 = encoder_attr_9.inverse_transform(predicted_labels[8])  # For Attribute 9
reversed_attr_10 = encoder_attr_10.inverse_transform(predicted_labels[9])  # For Attribute 10


In [44]:
# Prepare the submission DataFrame
submission_df.iloc[:predicted_values, 3] = reversed_attr_1
submission_df.iloc[:predicted_values, 4] = reversed_attr_2
submission_df.iloc[:predicted_values, 5] = reversed_attr_3
submission_df.iloc[:predicted_values, 6] = reversed_attr_4
submission_df.iloc[:predicted_values, 7] = reversed_attr_5
submission_df.iloc[:predicted_values, 8] = reversed_attr_6
submission_df.iloc[:predicted_values, 9] = reversed_attr_7
submission_df.iloc[:predicted_values, 10] = reversed_attr_8
submission_df.iloc[:predicted_values, 11] = reversed_attr_9
submission_df.iloc[:predicted_values, 12] = reversed_attr_10

# Fill the remaining rows with NaN
submission_df.iloc[predicted_values:] = np.nan

/tmp/ipykernel_30/4084899079.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['multicolor' 'multicolor' 'multicolor' ... 'dummy_value' 'dummy_value'
 'dummy_value']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  submission_df.iloc[:predicted_values, 3] = reversed_attr_1
/tmp/ipykernel_30/4084899079.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['polo' 'polo' 'polo' ... 'zari' 'zari' 'zari']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  submission_df.iloc[:predicted_values, 4] = reversed_attr_2
/tmp/ipykernel_30/4084899079.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['solid' 'solid' 'solid' ... 'small border' 'small border' 'small bord

In [45]:
submission_df=submission_df.iloc[:30205,:]

In [46]:
submission_df

,id,Category,len,Attribute_1,Attribute_2,Attribute_3,Attribute_4,Attribute_5,Attribute_6,Attribute_7,Attribute_8,Attribute_9,Attribute_10
0,0.0,Men Tshirts,5.0,multicolor,polo,solid,default,short sleeves,dummy_value,dummy_value,dummy_value,dummy_value,dummy_value
1,1.0,Men Tshirts,5.0,multicolor,polo,solid,solid,short sleeves,dummy_value,dummy_value,dummy_value,dummy_value,dummy_value
2,2.0,Men Tshirts,5.0,multicolor,polo,solid,solid,short sleeves,dummy_value,dummy_value,dummy_value,dummy_value,dummy_value
3,3.0,Men Tshirts,5.0,multicolor,dummy_value,solid,solid,short sleeves,dummy_value,dummy_value,dummy_value,dummy_value,dummy_value
4,4.0,Men Tshirts,5.0,multicolor,dummy_value,solid,solid,short sleeves,dummy_value,dummy_value,dummy_value,dummy_value,dummy_value
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
submission_df.fillna('nu', inplace=True)
        

/tmp/ipykernel_30/3818306898.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nu' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  submission_df.fillna('nu', inplace=True)


In [48]:
submission_df.iloc[:30205,0] = test_text.iloc[:,0]
submission_df.iloc[:30205,1] = test_text.iloc[:,1]

In [49]:
# Rename columns
submission_df.rename(columns={
    'Attribute_1': 'attr_1',
    'Attribute_2': 'attr_2',
    'Attribute_3': 'attr_3',
    'Attribute_4': 'attr_4',
    'Attribute_5': 'attr_5',
    'Attribute_6': 'attr_6',
    'Attribute_7': 'attr_7',
    'Attribute_8': 'attr_8',
    'Attribute_9': 'attr_9',
    'Attribute_10': 'attr_10',
}, inplace=True)

In [50]:
submission_df

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10
0,0,Men Tshirts,5.0,multicolor,polo,solid,default,short sleeves,dummy_value,dummy_value,dummy_value,dummy_value,dummy_value
1,1,Men Tshirts,5.0,multicolor,polo,solid,solid,short sleeves,dummy_value,dummy_value,dummy_value,dummy_value,dummy_value
2,2,Men Tshirts,5.0,multicolor,polo,solid,solid,short sleeves,dummy_value,dummy_value,dummy_value,dummy_value,dummy_value
3,3,Men Tshirts,5.0,multicolor,dummy_value,solid,solid,short sleeves,dummy_value,dummy_value,dummy_value,dummy_value,dummy_value
4,4,Men Tshirts,5.0,multicolor,dummy_value,solid,solid,short sleeves,dummy_value,dummy_value,dummy_value,dummy_value,dummy_value
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30200,30484,Women Tops & Tunics,nu,nu,nu,nu,nu,nu,nu,nu,nu,nu,nu
30201,30485,Women Tops & Tunics,nu,nu,nu,nu,nu,nu,nu,nu,nu,nu,nu
30202,30486,Women Tops & Tunics,nu,nu,nu,nu,nu,nu,nu,nu,nu,nu,nu
30203,30487,Women Tops & Tunics,nu,nu,nu,nu,nu,nu,nu,nu,nu,nu,nu


In [51]:
# Save the DataFrame to a CSV file
submission_file = 'submission_23k_21k.csv'
submission_df.to_csv(submission_file, index=False)

print("Submission file created successfully!")


Submission file created successfully!
